In [2]:
import itk

# Get the number of threads being used by ITK
num_threads = itk.MultiThreaderBase.GetGlobalDefaultNumberOfThreads()
print(f"ITK is using {num_threads/2} threads.")

ITK is using 64.0 threads.


In [1]:
import ants

In [2]:
import numpy as np
from scipy.ndimage import gaussian_filter
import ants

def create_elongated_blob_image(size, centers, radii, elongation_factors):
    image = np.zeros(size)
    for center, radius, elongation in zip(centers, radii, elongation_factors):
        zz, yy, xx = np.ogrid[:size[0], :size[1], :size[2]]
        # Apply elongation by scaling coordinates
        zz_scaled = zz / elongation[0]
        yy_scaled = yy / elongation[1]
        xx_scaled = xx / elongation[2]
        mask = (zz_scaled - center[0])**2 + (yy_scaled - center[1])**2 + (xx_scaled - center[2])**2 <= radius**2
        image[mask] = 1
    # Apply Gaussian smoothing to make blobs more realistic
    image = gaussian_filter(image, sigma=radius / 2)
    return ants.from_numpy(image.astype(np.float32))

# Image size and parameters
image_size = (200, 200, 200)
blob_radius = 10  # You can adjust the radius as needed

# Define centers 50 pixels apart
centers1 = [(100, 25, 100)]
centers2 = [(25, 100, 100)]
centers3 = [(100, 100, 25)]

# Define elongation factors for different axes
elongation1 = [(1.0, 2.0, 1.0)]  # Elongated along the Y axis
elongation2 = [(2.0, 1.0, 1.0)]  # Elongated along the X axis
elongation3 = [(1.0, 1.0, 2.0)]  # Elongated along the Z axis

# Create 3 images with elongated blobs in different locations
image1 = create_elongated_blob_image(image_size, centers=centers1, radii=[blob_radius], elongation_factors=elongation1)
image2 = create_elongated_blob_image(image_size, centers=centers2, radii=[blob_radius], elongation_factors=elongation2)
image3 = create_elongated_blob_image(image_size, centers=centers3, radii=[blob_radius], elongation_factors=elongation3)

# Save images for later inspection
ants.image_write(image1, "image1.nii.gz")
ants.image_write(image2, "image2.nii.gz")
ants.image_write(image3, "image3.nii.gz")

# Visual check (optional)
print("Image 1 sum:", np.sum(image1.numpy()))
print("Image 2 sum:", np.sum(image2.numpy()))
print("Image 3 sum:", np.sum(image3.numpy()))


Image 1 sum: 8309.0
Image 2 sum: 8308.999
Image 3 sum: 8309.0


In [3]:
import ants

# Read the created images (already in memory)
images = [image1, image2, image3]

# List to store affine-transformed images
affine_transformed_images = []

# Perform affine registration aon each image
for image in images:
    # Perform affine registration
    fixed_image = images[0]  # Using the first image as the fixed reference
    reg_affine = ants.registration(fixed=fixed_image, moving=image, type_of_transform="Rigid", aff_metric='CC', verbose=True,aff_sampling=2, aff_iterations=[20,0,0,0])

    # Apply the affine transformation to the moving image
    affine_transformed_image = reg_affine['warpedmovout']
    affine_transformed_images.append(affine_transformed_image)

    # Save the affine-transformed image for later inspection
    ants.image_write(affine_transformed_image, f"affine_transformed_image_{len(affine_transformed_images)}.nii.gz")


antsRegistration -d 3 -r [0x14e98a809188,0x14e98a809268,1] -m CC[0x14e98a809188,0x14e98a809268,1,2,regular,0.2] -t Rigid[0.25] -c 20x0x0x0 -s 3x2x1x0 -f 6x4x2x1 -u 1 -z 1 -o [/tmp/tmpt5ay8yqi,0x14e9cb7b3cc8,0x14e98a8092a8] -x [NA,NA] --float 1 --write-composite-transform 0 -v 1
All_Command_lines_OK
Using single precision for computations.
The composite transform comprises the following transforms (in order): 
  1. Center of mass alignment using fixed image: 0x14e98a809188 and moving image: 0x14e98a809268 (type = Euler3DTransform)
  Reading mask(s).
    Registration stage 0
      No fixed mask
      No moving mask
  number of levels = 4
  fixed image: 0x14e98a809188
  moving image: 0x14e98a809268
Dimension = 3
Number of stages = 1
Use histogram matching = true
Winsorize image intensities = false
  Lower quantile = 0
  Upper quantile = 1


Stage 1 State
   Image metric = CC
     Fixed image = Image (0x55b5dc03cdb0)
  RTTI typeinfo:   itk::Image<float, 3u>
  Reference Count: 2
  Modified 

In [5]:
!pwd

/groups/spruston/home/moharb/Unbiased/pyUnBiased


In [10]:
import os
import subprocess
import random
import numpy as np

# Paths to the images (replace these with your actual paths)
affine_transformed_images = [
    "affine_transformed_image_1.nii.gz",
    "affine_transformed_image_2.nii.gz",
    "affine_transformed_image_3.nii.gz"
]

# Parameters
max_iterations = 5  # Set the maximum number of iterations
output_dir = "/groups/spruston/home/moharb/Unbiased/pyUnBiased/OUTPUT"
python_script_path = "/groups/spruston/home/moharb/Unbiased/pyUnBiased/registration_script.py"  # Path to the registration script
python_env_activate = "/groups/spruston/home/moharb/mambaforge/envs/pyants/lib/python3.11/venv/scripts/common/activate"  # Path to your Python environment activation script

# Track total deformation strength for each iteration
total_deformation_strengths = []

# Iterate through the registration process
for iteration in range(max_iterations):
    print(f"Starting iteration {iteration + 1}...")
    
    job_ids = []
    iteration_strength = 0

    # Controlled pair selection: each image is the moving image only once
    remaining_images = list(range(len(affine_transformed_images)))
    random.shuffle(remaining_images)
    pairs = []

    for moving_index in remaining_images:
        possible_fixed_indices = [idx for idx in range(len(affine_transformed_images)) if idx != moving_index]
        fixed_index = random.choice(possible_fixed_indices)
        pairs.append((fixed_index, moving_index))

    # Submit jobs for each pair in this iteration
    for fixed_index, moving_index in pairs:
        print((fixed_index,moving_index))
        fixed_image = affine_transformed_images[fixed_index]
        moving_image = affine_transformed_images[moving_index]
        output_prefix = os.path.join(output_dir, f"pair_{fixed_index}_{moving_index}_iter_{iteration + 1}")

        # Prepare the bsub command with all necessary directives
        bsub_command = (
            f"bsub "
            f"-J reg_pair_{fixed_index}_{moving_index}_iter_{iteration + 1} "  # Job name
            f"-o {output_prefix}.%J.out "                                      # Output file
            f"-e {output_prefix}.%J.err "                                      # Error file
            f"-n 1 "                                                           # Number of cores
            f"-R 'rusage[mem=4096]' "                                          # Memory requirement
            f"bash -c 'source {python_env_activate} && python {python_script_path} "
            f"--fixed_image {fixed_image} --moving_image {moving_image} --output_prefix {output_prefix}'"
        )

        # Submit the job using subprocess and capture the job ID
        result = subprocess.run(bsub_command, shell=True, capture_output=True, text=True)
        print(result)
        job_id = result.stdout.strip().split('<')[1].split('>')[0]
        job_ids.append(job_id)
    
    # Wait for all jobs in this iteration to complete
    wait_command = f"bwait -w 'ended({','.join(job_ids)})'"
    subprocess.run(wait_command, shell=True)

    # After all jobs are completed, average the iteration strength and update images
    for fixed_index, moving_index in pairs:
        output_prefix = os.path.join(output_dir, f"pair_{fixed_index}_{moving_index}_iter_{iteration + 1}")
        
        # Read the deformation strength after the job finishes
        deformation_strength_file = f"{output_prefix}_deformation_strength.txt"
        with open(deformation_strength_file, "r") as f:
            iteration_strength += float(f.read())
        
        # Load the warped image and update the corresponding moving image in the list
        warped_image_path = f"{output_prefix}_warped.nii.gz"
        affine_transformed_images[moving_index] = warped_image_path

    # Average deformation strength for the iteration
    average_iteration_strength = iteration_strength / len(pairs)
    total_deformation_strengths.append(average_iteration_strength)
    print(f"Average Deformation Strength for Iteration {iteration + 1}: {average_iteration_strength}")

    # Optionally, check if deformation strength is below a threshold to stop early
    # if average_iteration_strength < some_threshold:
    #     print(f"Stopping early at iteration {iteration + 1} due to low deformation strength.")
    #     break

print("All iterations completed.")
np.savetxt(os.path.join(output_dir, "total_deformation_strengths.txt"), total_deformation_strengths)


Starting iteration 1...
(0, 1)
CompletedProcess(args="bsub -J reg_pair_0_1_iter_1 -o /groups/spruston/home/moharb/Unbiased/pyUnBiased/OUTPUT/pair_0_1_iter_1.%J.out -e /groups/spruston/home/moharb/Unbiased/pyUnBiased/OUTPUT/pair_0_1_iter_1.%J.err -n 1 -R 'rusage[mem=4096]' bash -c 'source /groups/spruston/home/moharb/mambaforge/envs/pyants/lib/python3.11/venv/scripts/common/activate && python /groups/spruston/home/moharb/Unbiased/pyUnBiased/registration_script.py --fixed_image affine_transformed_image_1.nii.gz --moving_image affine_transformed_image_2.nii.gz --output_prefix /groups/spruston/home/moharb/Unbiased/pyUnBiased/OUTPUT/pair_0_1_iter_1'", returncode=0, stdout='Job <140964130> is submitted to default queue <local>.\n', stderr='This job will be billed to spruston\n')
(0, 2)
CompletedProcess(args="bsub -J reg_pair_0_2_iter_1 -o /groups/spruston/home/moharb/Unbiased/pyUnBiased/OUTPUT/pair_0_2_iter_1.%J.out -e /groups/spruston/home/moharb/Unbiased/pyUnBiased/OUTPUT/pair_0_2_iter_1.%

ended(140964130,140964131,140964132): Wait condition syntax error


FileNotFoundError: [Errno 2] No such file or directory: '/groups/spruston/home/moharb/Unbiased/pyUnBiased/OUTPUT/pair_0_1_iter_1_deformation_strength.txt'

In [8]:
job_ids

['140964123', '140964124', '140964125']

In [ ]:
import ants
import os

# Parameters: paths to the target image, moving images, and output directory
target_image_path = '/path/to/target_image.nii.gz'  # Replace with your target image path
moving_images_paths = [
    '/path/to/moving_image1.nii.gz',
    '/path/to/moving_image2.nii.gz'
    # Add more moving images as needed
]
output_dir = '/path/to/output_directory'  # Replace with your output directory

# Load the target image
target_image = ants.image_read(target_image_path)

# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Perform affine registration for each moving image
for moving_image_path in moving_images_paths:
    # Load the moving image
    moving_image = ants.image_read(moving_image_path)
    
    # Load the corresponding mask (same file name with '_Probabilities.tiff' suffix)
    mask_path = moving_image_path.replace('.nii.gz', '_Probabilities.tiff')
    if os.path.exists(mask_path):
        mask_image = ants.image_read(mask_path)
        print(f"Mask found and loaded for: {moving_image_path}")
    else:
        mask_image = None
        print(f"No mask found for: {moving_image_path}. Proceeding without mask.")
    
    # Perform affine registration with mask (if mask is available)
    if mask_image is not None:
        reg_affine = ants.registration(fixed=target_image, moving=moving_image, type_of_transform='Affine', mask=mask_image)
    else:
        reg_affine = ants.registration(fixed=target_image, moving=moving_image, type_of_transform='Affine')
    
    # Construct output path
    moving_image_filename = os.path.basename(moving_image_path)
    output_path = os.path.join(output_dir, f"affine_{moving_image_filename}")
    
    # Save the registered image
    ants.image_write(reg_affine['warpedmovout'], output_path)
    print(f"Registered image saved to: {output_path}")
